# Cardiovascular desease detection

## Project by: PARAS VARSHNEY

### Importing Libraries

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.externals import joblib 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [63]:
data_url = "https://raw.githubusercontent.com/paras009/Cardiovascular-Disease-Detection/master/cardio_train.csv?token=AGEPDXH47ZCA3MSUDB6K7I26XQPH2"

In [43]:
# raw_df = pd.read_csv(data_url, sep=";")
raw_df = pd.read_csv("cardio_train.csv", sep=";")

In [44]:
raw_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,988,22469,1,155,69.0,130,80,2,2,0,0,1,0
1,989,14648,1,163,71.0,110,70,1,1,0,0,1,1
2,990,21901,1,165,70.0,120,80,1,1,0,0,1,0
3,991,14549,2,165,85.0,120,80,1,1,1,1,1,0
4,992,23393,1,155,62.0,120,80,1,1,0,0,1,0


In [45]:
raw_df.columns

Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
       'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [46]:
raw_df.shape

(69301, 13)

In [47]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69301 entries, 0 to 69300
Data columns (total 13 columns):
id             69301 non-null int64
age            69301 non-null int64
gender         69301 non-null int64
height         69301 non-null int64
weight         69301 non-null float64
ap_hi          69301 non-null int64
ap_lo          69301 non-null int64
cholesterol    69301 non-null int64
gluc           69301 non-null int64
smoke          69301 non-null int64
alco           69301 non-null int64
active         69301 non-null int64
cardio         69301 non-null int64
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [48]:
raw_df.isna().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [49]:
# raw_df.to_csv(r'exported_cardiovascular.csv', index = False)

In [54]:
df_X = raw_df.drop(["id", "cardio"], axis=1)
df_y = raw_df.loc[:, "cardio"]

In [51]:
raw_df.cardio.value_counts()

0    34679
1    34622
Name: cardio, dtype: int64

### Feature Engineering

##### converting log normal to normal

<hr>
<hr>

In [66]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = df_X.shape[1]))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [67]:
# classifier.fit(df_X, df_y, batch_size = 10, epochs = 10)


Epoch 1/10
69301/69301 [==============================] - 26s 378us/step - loss: 0.6930 - accuracy: 0.5085
Epoch 2/10
69301/69301 [==============================] - 25s 359us/step - loss: 0.6931 - accuracy: 0.5017
Epoch 3/10
  940/69301 [..............................] - ETA: 28s - loss: 0.6928 - accuracy: 0.5298

KeyboardInterrupt: 